In [1]:
from glob import glob
import json
import re
import pandas as pd

In [2]:
zveza_all = []
for x in glob("./data/scraped/cpp_in__co_ali_go__in_vs/*.json"):
    with open(x, "r") as f:
        data = json.load(f)

    zveza_all.append((data["zveza"], data["meta"]["Evidenčna številka"]))


In [3]:
set([y for x in zveza_all for y in x[0].keys()])

{'',
 'EU - Direktive, Uredbe, Sklepi / Odločbe, Sporazumi, Pravila',
 'Konvencije, Deklaracije Resolucije',
 'Mednarodne Pogodbe',
 'Podzakonski akti / Vsi drugi akti',
 'Predpisi drugih držav',
 'RS - Ustava, Zakoni, Sporazumi, Pogodbe',
 'Uzance',
 'Without category'}

In [4]:
zveza_unformatted = set([(*y, x[1]) for x in zveza_all for y in x[0].items()]) - set(
    [""]
)
zveza_unformatted = pd.DataFrame(
    zveza_unformatted, columns=["category", "zveza", "evidencna_stevilka"]
)
zveza_unformatted

,category,zveza,evidencna_stevilka
0,Without category,"URS člen 32. EKČP člen 6.ZPP člen 63, 67.",VS09421
1,Without category,ZPP člen 67.,VS07999
2,Without category,"ZPP člen 339, 339/2, 339/2-8, 339/2-14. ZZK-1 ...",VS0018712
3,Without category,"URS člen 14, 22, 23. ZZZDR člen 59.",VS0017599
4,"RS - Ustava, Zakoni, Sporazumi, Pogodbe",Zakon o izvršbi in zavarovanju (1998) - ZIZ -...,VS00054720
...,...,...,...
16531,Without category,ZPP člen 67.,VS0013684
16532,"RS - Ustava, Zakoni, Sporazumi, Pogodbe",Zakon o izvršbi in zavarovanju (1998) - ZIZ -...,VS00061100
16533,Without category,"ZOR člen 178, 178/2, 200, 200/1, 200/2, 203.ZP...",VS0010818
16534,"RS - Ustava, Zakoni, Sporazumi, Pogodbe","Stvarnopravni zakonik (2002) - SPZ - člen 48,...",VS00045062


# ZAKONI RS

In [5]:
def split_RS(zveza):
    zveza = re.sub(r"[\n\t\r]", "", zveza)

    split_phrase = "#$%^"
    pattern = r"[A-Z][\w\s]+\(\d+\)"
    # zveza_splitters = [".", "Konvencija ", "Direktiva ", "Uredba ", "Pravilnik "]

    matches = re.findall(pattern, zveza)
    matches.append("Zakon o")

    for match in matches:
        zveza = zveza.replace(match, f"{split_phrase}{match}")

    return [x.strip() for x in zveza.split(split_phrase) if x]


def _format_clen_from_docs(clen):
    if clen.isdigit():
        return f"{clen}. člen"
    digits = "".join([x for x in clen if x.isdigit()])
    alphas = "".join([x for x in clen if x.isalpha()])
    return f"{digits}.{alphas} člen"


formatted_data = []
for _val in zveza_unformatted.loc[
    zveza_unformatted["category"] == "RS - Ustava, Zakoni, Sporazumi, Pogodbe",
    ["zveza", "evidencna_stevilka"],
].values:
    zveza, evidencna_stevilka = _val
    for z in split_RS(zveza):
        try:
            _tmp = z.split(" - ")
            if len(_tmp) == 3:
                title, abbr, clens_unformatted = _tmp
            else:
                title, clens_unformatted = _tmp
                abbr = ""

            title, abbr, clens_unformatted = (
                title.strip(),
                abbr.strip(),
                clens_unformatted.strip(),
            )
            title = re.sub(r"\(\d+\)", "", title)
        except:
            print("ERROR split: ", z)
            continue

        try:
            zveza_true = []
            for clen in clens_unformatted.split(","):
                clen = clen.strip()
                clen = clen.split("/")[0]
                clen = clen.split("-")[0]
                if _split := clen.split(" "):
                    clen = [x for x in _split if not all(y.isalpha() for y in x)][0]
                zveza_true.append(clen)

            zveza_true = sorted(
                [f"{_format_clen_from_docs(x)} {abbr}" for x in list(set(zveza_true))],
                key=lambda x: int(x.split(".")[0]),
            )
            formatted_data.append(
                {
                    "title": title,
                    "abbr": abbr,
                    "zveza": zveza_true,
                    "evidencna_stevilka": evidencna_stevilka,
                }
            )
        except:
            print("ERROR formatting: ", zveza)
            continue


ERROR split:  E
ERROR split:  Ustava Republike Slovenije (1991) - URS  - člen 155 Pomorski zakonik - PZ  - člen 44, 44/2
ERROR formatting:  Zakon o pravdnem postopku (1999) - ZPP  - člen 190, 190/2, 337, 337/1, 370, 370/3,  Stvarnopravni zakonik (2002) - SPZ  - člen 146, 146/4, 153, 153/1 Zakon o ukrepih Republike Slovenije za krepitev stabilnosti bank (2012) - ZUKSB  - člen 11, 11/1, 11/7, 11/8, 29, 29/2 Zakon o izvršbi in zavarovanju (1998) - ZIZ  - člen 17
ERROR split:  Zakon o državljanstvu oseb na območju, priključenem po mirovni pogodbi z
ERROR formatting:  Zakon o pravdnem postopku (1999) - ZPP  - člen 105a,  Zakon o sodnih taksah (2008) - ZST-1  - člen 8, 16, 18, 18/5, 19
ERROR split:  Stanovanjski zakon (1991) - SZ  - člen 117 - 123
ERROR split:  Zakon o pravdnem postopku (1999) - ZPP  - člen 341, 367a, 367a/1, 367c, 367c/2 Obligacijski zakonik (uradno prečiščeno besedilo) (2007) - OZ-UPB1  - člen 186, 186/1
ERROR split:  ZPP-
ERROR split:  E
ERROR split:  Zakon o pravdnem pos

In [6]:
RS_cleaned_data = pd.DataFrame(formatted_data)
RS_cleaned_data

,title,abbr,zveza,evidencna_stevilka
0,Zakon o izvršbi in zavarovanju,ZIZ,[10. člen ZIZ],VS00054720
1,Zakon o pravdnem postopku,ZPP,[377. člen ZPP],VS00054720
2,Zakon o industrijski lastnini,ZIL-1,"[48. člen ZIL-1, 116. člen ZIL-1]",VS00024662
3,Zakon o pravdnem postopku,ZPP,[181. člen ZPP],VS00024662
4,Zakon o pravdnem postopku,ZPP,"[367.c člen ZPP, 367.a člen ZPP]",VS00030021
...,...,...,...,...
10690,Zakon o izvršbi in zavarovanju,ZIZ,[10. člen ZIZ],VS00061100
10691,Zakon o pravdnem postopku,ZPP,[377. člen ZPP],VS00061100
10692,Stvarnopravni zakonik,SPZ,[48. člen SPZ],VS00045062
10693,Obligacijski zakonik,OZ,"[52. člen OZ, 58. člen OZ]",VS00045062


# WITHOUT CATEGORY

In [7]:
def split_uncatagorized(zveza):
    zveza = re.sub(r"[\n\t\r]", "", zveza)
    zveza = zveza.replace("ZPP (1977)", "ZPP")

    split_phrase = "#$%^"
    pattern = r"[\w\d-]+ člen"

    matches = re.findall(pattern, zveza)

    for match in matches:
        zveza = zveza.replace(match, f"{split_phrase}{match}")

    return [x.strip() for x in zveza.split(split_phrase) if x]


formatted_data = []
for x in zveza_unformatted.loc[
    zveza_unformatted["category"] == "Without category",
    ["zveza", "evidencna_stevilka"],
].values:
    zveza, evidencna_stevilka = x
    zveza = split_uncatagorized(zveza)

    for z in zveza:
        pattern = r"[\w\d-]+ člen"
        matches = re.findall(pattern, z)

        if len(matches) != 1:
            print("ERROR MATCH: ", z, matches)
            continue

        abbr = matches[0].replace("člen", "").strip()
        z = z.replace(matches[0], "")
        zveza_split = z.split(",")
        zveza_split = [
            x.strip().strip(".").split("/")[0].split("-")[0] for x in zveza_split if x
        ]

        try:
            zveza_true = sorted(
                [f"{_format_clen_from_docs(x)} {abbr}" for x in list(set(zveza_split))],
                key=lambda x: int(x.split(".")[0]),
            )
        except:
            print("ERROR SPLIT: ", zveza_split)
            continue

        formatted_data.append(
            {
                "title": "Without category",
                "abbr": abbr,
                "zveza": zveza_true,
                "evidencna_stevilka": evidencna_stevilka,
            }
        )
WC_cleaned_data = pd.DataFrame(formatted_data)
WC_cleaned_data


ERROR MATCH:  Uredba (ES) št. 864/2007 Evropskega parlamenta in Sveta z dne 11. julija 2007 o pravu, ki se uporablja za nepogodbene obveznosti (Uredba Rim II) člen 4, 4-1, 5, 5-1, 5-1/1. []
ERROR MATCH:  ZPPčlen 214, 244, 247. []
ERROR MATCH:  ZPPčlen 339, 339/2-14. []
ERROR SPLIT:  ['40c', '40', '40', 'vrstah izvršb in poteku avtomatiziranega izvršilnega']
ERROR MATCH:  ZM []
ERROR MATCH:  ZM []
ERROR SPLIT:  ['125. Navodilo o tem', 'kaj se šteje za dokumentacijo za prenos kmetijskih zemljišč', 'kmetij in gozdov na Sklad kmetijskih zemljišč in gozdov Republike Slovenije oziroma na občine točka 5', '5', '5']
ERROR MATCH:  ZPP. []
ERROR MATCH:  Zakon o celotnem prihodku in []
ERROR MATCH:  ODZ paragraf 169, 551, 754. []
ERROR MATCH:  Uredba Sveta (ES) št. 1348/2000 z dne 29. 9. 2000 o vročanju                     sodnih in zunajsodnih pisanj v civilnih ali gospodarskih zadevah                     v državah []
ERROR SPLIT:  ['74', '74', '74', 'kaj se šteje za dokumentacijo za prenos kmet

,title,abbr,zveza,evidencna_stevilka
0,Without category,URS,[32. člen URS],VS09421
1,Without category,EKČP,[6. člen EKČP],VS09421
2,Without category,ZPP,"[63. člen ZPP, 67. člen ZPP]",VS09421
3,Without category,ZPP,[67. člen ZPP],VS07999
4,Without category,ZPP,[339. člen ZPP],VS0018712
...,...,...,...,...
16861,Without category,ZPP,[67. člen ZPP],VS0013684
16862,Without category,ZOR,"[178. člen ZOR, 200. člen ZOR, 203. člen ZOR]",VS0010818
16863,Without category,ZPP,"[41. člen ZPP, 367. člen ZPP, 377. člen ZPP, 3...",VS0010818
16864,Without category,ZPP,[367.a člen ZPP],VS4002326


EU

In [8]:
eu_types = [
    "EU - Direktive, Uredbe, Sklepi / Odločbe, Sporazumi, Pravila",
]

formatted_data = []
for x in zveza_unformatted.loc[
    zveza_unformatted["category"].isin(eu_types),
    ["zveza", "evidencna_stevilka"],
].values:
    zveza, evidencna_stevilka = x
    if "Direktiva" in zveza and not "Uredba" in zveza:
        zveza, evidencna_stevilka = x

        for _direktiva in [
            f"Direktiva{x}" for x in zveza.split("Direktiva") if x.strip()
        ]:
            title, clens_unformatted = _direktiva.split(" - ")
            pattern = r"\d{1,2}/\d{1,2}/EGS"
            matches = re.findall(pattern, title)

            if len(matches) != 1:
                print("ERROR MATCH: ", _direktiva)
                continue
            abbr = matches[0]

            clens_unformatted = clens_unformatted.replace("člen", "")
            clens_split = [
                x.strip().strip(".").split("/")[0].split("-")[0]
                for x in clens_unformatted.split(",")
            ]

            try:
                zveza_true = sorted(
                    [
                        f"{_format_clen_from_docs(x)} {abbr}"
                        for x in list(set(clens_split))
                    ],
                    key=lambda x: int(x.split(".")[0]),
                )
            except:
                print("ERROR SPLIT: ", clens_split)
                continue

            formatted_data.append(
                {
                    "title": title,
                    "abbr": abbr,
                    "zveza": zveza_true,
                    "evidencna_stevilka": evidencna_stevilka,
                }
            )
EU_cleaned_data = pd.DataFrame(formatted_data)
EU_cleaned_data


,title,abbr,zveza,evidencna_stevilka
0,Direktiva Sveta 93/13/EGS z dne 5. aprila 1993...,93/13/EGS,[3. člen 93/13/EGS],VS00066953
1,Direktiva Sveta 93/13/EGS z dne 5. aprila 1993...,93/13/EGS,"[4. člen 93/13/EGS, 5. člen 93/13/EGS]",VS00078699
2,Direktiva Sveta 93/13/EGS z dne 5. aprila 1993...,93/13/EGS,[3. člen 93/13/EGS],VS00079849
3,Direktiva Sveta 93/13/EGS z dne 5. aprila 1993...,93/13/EGS,[3. člen 93/13/EGS],VS00077796
4,Direktiva Sveta 93/13/EGS z dne 5. aprila 1993...,93/13/EGS,"[3. člen 93/13/EGS, 8. člen 93/13/EGS]",VS00081811
5,Direktiva Sveta 93/13/EGS z dne 5. aprila 1993...,93/13/EGS,"[1. člen 93/13/EGS, 4. člen 93/13/EGS, 5. člen...",VS00076573
6,Direktiva Sveta 93/13/EGS z dne 5. aprila 1993...,93/13/EGS,"[4. člen 93/13/EGS, 5. člen 93/13/EGS]",VS00075923
7,Direktiva Sveta 93/13/EGS z dne 5. aprila 1993...,93/13/EGS,"[4. člen 93/13/EGS, 5. člen 93/13/EGS]",VS00075522
8,Direktiva Sveta 93/13/EGS z dne 5. aprila 1993...,93/13/EGS,[3. člen 93/13/EGS],VS00076320
9,Direktiva Sveta 93/13/EGS z dne 5. aprila 1993...,93/13/EGS,[3. člen 93/13/EGS],VS00051585


In [9]:
eu_types = [
    "EU - Direktive, Uredbe, Sklepi / Odločbe, Sporazumi, Pravila",
]

formatted_data = []
for x in zveza_unformatted.loc[
    zveza_unformatted["category"].isin(eu_types),
    ["zveza", "evidencna_stevilka"],
].values:
    zveza, evidencna_stevilka = x
    if "Uredba" in zveza and not "Direktiva" in zveza:
        zveza, evidencna_stevilka = x

        for _uredba in [f"Uredba{x}" for x in zveza.split("Uredba") if x.strip()]:
            print(_uredba)
            title, clens_unformatted = _uredba.split(" - ")
            pattern = r"\d{3,4}/\d{4}"
            matches = re.findall(pattern, title)

            if not matches:
                pattern = r"\d{1,2}/\d{4}"
                matches = re.findall(pattern, title)

            if len(matches) != 1:
                print("ERROR MATCH: ", _uredba)
                continue
            abbr = matches[0]

            clens_unformatted = clens_unformatted.replace("člen", "")
            clens_split = [
                x.strip().strip(".").split("/")[0].split("-")[0]
                for x in clens_unformatted.split(",")
            ]

            try:
                zveza_true = sorted(
                    [
                        f"{_format_clen_from_docs(x)} {abbr}"
                        for x in list(set(clens_split))
                    ],
                    key=lambda x: int(x.split(".")[0]),
                )
            except:
                print("ERROR SPLIT: ", clens_split)
                continue

            formatted_data.append(
                {
                    "title": title,
                    "abbr": abbr,
                    "zveza": zveza_true,
                    "evidencna_stevilka": evidencna_stevilka,
                }
            )
formatted_data = pd.DataFrame(formatted_data)
formatted_data


Uredba (EU) št. 1215/2012 Evropskega parlamenta in Sveta z dne 12. decembra 2012 o pristojnosti in priznavanju ter izvrševanju sodnih odločb v civilnih in gospodarskih zadevah - člen 45, 45/1, 45/1-a, 45/1-b
Uredba (EU) št. 1215/2012 Evropskega parlamenta in Sveta z dne 12. decembra 2012 o pristojnosti in priznavanju ter izvrševanju sodnih odločb v civilnih in gospodarskih zadevah - člen 45, 45/1-a, 45/1-b
Uredba (EU) št. 1215/2012 Evropskega parlamenta in Sveta z dne 12. decembra 2012 o pristojnosti in priznavanju ter izvrševanju sodnih odločb v civilnih in gospodarskih zadevah - člen 8, 8/3, 25, 25/1
Uredba Sveta (ES) št. 44/2001 z dne 22. decembra 2000 o pristojnosti in priznavanju ter izvrševanju sodnih odločb v civilnih in gospodarskih zadevah - člen 15, 16, 17
Uredba (EU) št. 1215/2012 Evropskega parlamenta in Sveta z dne 12. decembra 2012 o pristojnosti in priznavanju ter izvrševanju sodnih odločb v civilnih in gospodarskih zadevah - člen 38, 43, 45, 45/1, 45/1-a, 45/1-b, 51
Ure

,title,abbr,zveza,evidencna_stevilka
0,Uredba (EU) št. 1215/2012 Evropskega parlament...,1215/2012,[45. člen 1215/2012],VS00033950
1,Uredba (EU) št. 1215/2012 Evropskega parlament...,1215/2012,[45. člen 1215/2012],VS00032833
2,Uredba (EU) št. 1215/2012 Evropskega parlament...,1215/2012,"[8. člen 1215/2012, 25. člen 1215/2012]",VS00008620
3,Uredba Sveta (ES) št. 44/2001 z dne 22. decemb...,44/2001,"[15. člen 44/2001, 16. člen 44/2001, 17. člen ...",VS00042901
4,Uredba (EU) št. 1215/2012 Evropskega parlament...,1215/2012,"[38. člen 1215/2012, 43. člen 1215/2012, 45. č...",VS00020952
5,Uredba (EU) št. 1215/2012 Evropskega parlament...,1215/2012,"[45. člen 1215/2012, 46. člen 1215/2012, 47. č...",VS00074606
6,Uredba Sveta (ES) št. 1215/1999 z dne 10. juni...,1215/1999,[45. člen 1215/1999],VS00062353
7,Uredba (EU) št. 1215/2012 Evropskega parlament...,1215/2012,"[8. člen 1215/2012, 25. člen 1215/2012]",VS00070341
8,Uredba (EU) št. 650/2012 Evropskega parlamenta...,650/2012,"[39. člen 650/2012, 43. člen 650/2012]",VS00076201
9,Uredba (EU) št. 1215/2012 Evropskega parlament...,1215/2012,"[7. člen 1215/2012, 25. člen 1215/2012, 26. čl...",VS00046130


OTHER

In [10]:
other_types = [
    "Konvencije, Deklaracije Resolucije",
    "Mednarodne Pogodbe",
    "Podzakonski akti / Vsi drugi akti",
    "Predpisi drugih držav",
]

formatted_data = []
for x in zveza_unformatted.loc[
    zveza_unformatted["category"].isin(other_types),
    ["zveza", "evidencna_stevilka"],
].values:
    zveza, evidencna_stevilka = x
    print(zveza)


Kodeks slovenskih novinarjev (2002) - člen 3, 17
Uredba o dopolnitvi uredbe o merilih za določanje oprostitev pri plačilih socialno varstvenih storitev (2003) - člen 27
Sodni red - člen 174
Konvencija o varstvu človekovih pravic in temeljnih svoboščin (EKČP)  - člen 6
Uredba o izdajanju in upoštevanju potrdil na podlagi neizplačanega dela neto osnovnih plač (1993) - člen 1, 6, 7, 8, 9, 10
Konvencija o varstvu človekovih pravic in temeljnih svoboščin (EKČP)  - člen 6
Sodni red (2016) - člen 159, 159/2
Konvencija o varstvu človekovih pravic in temeljnih svoboščin (EKČP) - člen 6
Pravilnik o poslovanju notarja (1994) - člen 28, 32, 34
Statut Mestne občine Ljubljana (uradno prečiščeno besedilo) (2021) - člen 51, 51-9
Pogodba o delovanju  Evropske unije (PDEU) - člen 102, 102/2, 102/2-c, 267, 267/3
Konvencija o pridobivanju dokazov v civilnih ali gospodarskih zadevah v tujini (Haaška konvencija) - člen 7, 11
Pravilnik o načinih valorizacije denarnih obveznosti, ki jih v večletnih pogodbah d

In [16]:
merged_data = pd.concat([RS_cleaned_data, WC_cleaned_data, EU_cleaned_data])
merged_data["title"] = merged_data["title"].str.replace("Without category", "")
merged_data.reset_index(drop=True).to_parquet(
    "./data/mappings/legislation_mappings_from_cpp_scraped.parquet"
)